# This is the Notebook to get the Data from the CBS API

In [1]:
import pandas as pd
import cbsodata as cbs
import re
from glob import glob
import pyreadstat
import numpy as np
import os

In [2]:
# This function takes the unique categories of the identifier columns, puts their values in new columns and deletes the original column

def split_column(df, values, category, region, year):
    if type(values) is list:
        for x in df[f'{category}'].unique():
            for v in values:
                df[f'{x}_{v}'] = np.where(df[f'{category}'] == x, df[f'{v}'], None)
        for v in values:
            df = df.drop([f'{v}'], axis=1)
    if type(values) is str:
        for x in df[f'{category}'].unique():
            df[f'{x}_{values}'] = np.where(df[f'{category}'] == x, df[f'{values}'], None)
        df = df.drop([f'{values}'], axis=1)
    df = df.groupby([f'{region}', f'{year}']).agg(max).drop([f'{category}'], axis=1)
    return df

In [3]:
# This function takes the unique categories of the identifier columns, puts their values in new columns and deletes the original column

def split_column2(df, values, category1, category2, region, year):
    if type(values) is list:
        for x in df[f'{category1}'].unique():
            for y in df[f'{category2}'].unique():
                for v in values:
                    df[f'{x}_{y}_{v}'] = np.where((df[f'{category2}'] == y) & (df[f'{category1}'] == x), df[f'{v}'], None)
        for v in values:
            df = df.drop([f'{v}'], axis=1)
    if type(values) is str:
        for x in df[f'{category1}'].unique():
            for y in df[f'{category2}'].unique():
                df[f'{x}_{y}_{values}'] = np.where((df[f'{category2}'] == y) & (df[f'{category1}'] == x), df[f'{values}'], None)
        df = df.drop([f'{values}'], axis=1)
    df = df.groupby([f'{region}', f'{year}']).agg(max).drop([f'{category1}', f'{category2}'], axis=1)
    return df

## Downloading Population number

In [44]:
metadata = pd.DataFrame(cbs.get_meta('37259ned', 'DataProperties'))
metadata

,odata.type,ID,Position,ParentID,Type,Key,Title,Description,MapYear,ReleasePolicy,Datatype,Unit,Decimals,Default,PresentationType
0,Cbs.OData.Dimension,0,0.0,NaN,Dimension,Geslacht,Geslacht,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cbs.OData.GeoDimension,1,1.0,NaN,GeoDimension,RegioS,Regio's,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cbs.OData.TimeDimension,2,2.0,NaN,TimeDimension,Perioden,Perioden,,NaN,True,NaN,NaN,NaN,NaN,NaN
3,Cbs.OData.Topic,3,3.0,NaN,Topic,BevolkingOp1Januari_1,Bevolking op 1 januari,Bevolking:\r\nDe inwoners van een bepaald gebi...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
4,Cbs.OData.Topic,4,4.0,NaN,Topic,Bevolkingsdichtheid_2,Bevolkingsdichtheid,Bevolking op 1 januari per km2 oppervlakte lan...,NaN,NaN,Long,aantal,0.0,Impossible,Relative
5,Cbs.OData.TopicGroup,5,NaN,NaN,TopicGroup,,Levend geboren kinderen,Levend geboren kind:\r\nKind dat na geboorte e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cbs.OData.Topic,6,5.0,5.0,Topic,LevendGeborenKinderen_3,Levend geboren kinderen,None,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
7,Cbs.OData.Topic,7,6.0,5.0,Topic,LevendGeborenKinderenRelatief_4,"Levend geboren kinderen, relatief",Levend geboren kinderen per duizend van de gem...,NaN,NaN,Double,promille,1.0,Impossible,Relative
8,Cbs.OData.TopicGroup,8,NaN,NaN,TopicGroup,,Overledenen,Overledene:\r\nPersoon die is overleden waarbi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Cbs.OData.Topic,9,7.0,8.0,Topic,Overledenen_5,Overledenen,None,NaN,NaN,Long,aantal,0.0,Impossible,Absolute


In [45]:
data = pd.DataFrame(cbs.get_data('37259ned', select = (['BevolkingOp1Januari_1', 'RegioS', 'Perioden', 'Geslacht'])))

In [47]:
val = 'BevolkingOp1Januari_1'
cat = 'Geslacht'
reg = 'RegioS'
year = 'Perioden'
data = split_column(data, val, cat, reg, year)

In [48]:
data.to_csv('Population.csv')

## Downloading Fertility Data

In [8]:
metadata = pd.DataFrame(cbs.get_meta('37201', 'Perioden'))
metadata

,Key,Title,Description,Status
0,1988JJ00,1988,None,Definitief
1,1989JJ00,1989,None,Definitief
2,1990JJ00,1990,None,Definitief
3,1991JJ00,1991,None,Definitief
4,1992JJ00,1992,None,Definitief
5,1993JJ00,1993,None,Definitief
6,1994JJ00,1994,None,Definitief
7,1995JJ00,1995,None,Definitief
8,1996JJ00,1996,None,Definitief
9,1997JJ00,1997,None,Definitief


In [35]:
data = pd.DataFrame(cbs.get_data('37201', select = (['LevendGeborenKinderenRelatief_2', 'RegioS', 'Perioden'])))

In [41]:
# Making RegioS and Perioden the index values as in other dataframes

data.index = pd.MultiIndex.from_arrays(data[['RegioS', 'Perioden']].values.T, names=['RegioS', 'Perioden'])
data = data.drop(['RegioS', 'Perioden'], axis=1)

KeyError: "None of [Index(['RegioS', 'Perioden'], dtype='object')] are in the [columns]"

In [40]:
data.to_csv('Fertility.csv')

## Downloading Unemployment Data

In [9]:
# Downloading data for unemployment
metadata = pd.DataFrame(cbs.get_meta('84961NED', 'Perioden'))
metadata

,Key,Title,Description,CategoryGroupID
0,2003JJ00,2003,,None
1,2004JJ00,2004,,None
2,2005JJ00,2005,,None
3,2006JJ00,2006,,None
4,2007JJ00,2007,,None
5,2008JJ00,2008,,None
6,2009JJ00,2009,,None
7,2010JJ00,2010,,None
8,2011JJ00,2011,,None
9,2012JJ00,2012,,None


In [40]:
data = pd.DataFrame(cbs.get_data('84961NED',
                                      select = ['Werkloosheidspercentage_13', 'RegioS',
                                                'Perioden', 'Persoonskenmerken']))
data = data[(data['Persoonskenmerken'] == 'Totaal personen') |
            (data['Persoonskenmerken'] == 'Geslacht: mannen') |
            (data['Persoonskenmerken'] == 'Geslacht: vrouwen')]

In [23]:
data['Geslacht: mannen_Werkloosheidspercentage_13'] = np.where(data['Persoonskenmerken'] == 'Geslacht: mannen', data['Werkloosheidspercentage_13'], None)
data = data.drop(['Werkloosheidspercentage_13'], axis=1)
data = data.groupby(['RegioS', 'Perioden']).agg(max).drop(['Persoonskenmerken'], axis=1)

In [42]:
val = 'Werkloosheidspercentage_13'
cat = 'Persoonskenmerken'
reg = 'RegioS'
year = 'Perioden'
data = split_column(data, val, cat, reg, year)

In [44]:
data.to_csv('Unemployment.csv')

## Download immigration change data

alternatives:

84974NED - current, from 2011

60056ned - current, from 1988

In [14]:
metadata = pd.DataFrame(cbs.get_meta('60056ned', 'Perioden'))
metadata

,Key,Title,Description,Status
0,1988JJ00,1988,None,Definitief
1,1989JJ00,1989,None,Definitief
2,1990JJ00,1990,None,Definitief
3,1991JJ00,1991,None,Definitief
4,1992JJ00,1992,None,Definitief
5,1993JJ00,1993,None,Definitief
6,1994JJ00,1994,None,Definitief
7,1995JJ00,1995,None,Definitief
8,1996JJ00,1996,None,Definitief
9,1997JJ00,1997,None,Definitief


In [6]:
print(metadata['Description'][4])

Migratie per duizend van de gemiddelde bevolking in het waarnemingsjaar.

Migratie:
Verhuizing van personen van of naar een ander woonadres.

Bevolking:
De inwoners van een bepaald gebied.
In de bevolkingsaantallen zijn uitsluitend personen begrepen die zijn opgenomen in het bevolkingsregister van een Nederlandse gemeente.
In principe wordt iedereen die voor onbepaalde tijd in Nederland woont, opgenomen in het bevolkingsregister van de woongemeente.
Personen die tot de bevolking van Nederland behoren, maar voor wie geen vaste woonplaats valt aan te wijzen, zijn opgenomen in het bevolkingsregister van de gemeente 's-Gravenhage.
In de bevolkingsregisters zijn niet opgenomen de in Nederland wonende personen waarvoor uitzonderingsregels gelden met betrekking tot opneming in de bevolkingsregisters (bijvoorbeeld diplomaten en NAVO militairen) en personen die niet legaal in Nederland verblijven.

Verwerking grenswijziging Rotterdam en Rozenburg in 2010
Per 18 maart 2010 is de gemeente Rozenbu

In [12]:
# for 60056ned

# take only migratiesaldo from Migratie
filters = ["Nationaliteit eq 'NAT9489'", # total non-dutch
           "Nationaliteit eq 'NAT9275'", # total EU excl. Dutch
           "Nationaliteit eq 'NAT9482'", # total Western excl. Dutch
           "Nationaliteit eq 'NAT9483'"] # total Non-Western

data = pd.DataFrame()
for x in filters:
    data = data.append(pd.DataFrame(cbs.get_data('60056ned',
                                          select = ['TotaalMannen_22', 'RegioS', 'Perioden',
                                                    'TotaalVrouwen_34', 'EuropeseUnieExclusiefNederland_81',
                                                   'TotaalNietWesters_87', 'OverigeWesterseLanden_95', 
                                                   'NietNederlands_66', 'NietNederlandseVrouwen_69'],
                                         filters =  x)))

In [13]:
data

,TotaalMannen_22,RegioS,Perioden,TotaalVrouwen_34,EuropeseUnieExclusiefNederland_81,TotaalNietWesters_87,OverigeWesterseLanden_95,NietNederlands_66,NietNederlandseVrouwen_69
0,48678.0,Nederland,1988,42560.0,15517.0,45300.0,24952.0,32019.0,26243.0
1,53782.0,Nederland,1989,45132.0,15645.0,50505.0,26773.0,36699.0,28686.0
2,65336.0,Nederland,1990,52014.0,18578.0,61483.0,32368.0,46995.0,34269.0
3,66888.0,Nederland,1991,53361.0,20164.0,60609.0,35930.0,48379.0,35958.0
4,63141.0,Nederland,1992,53785.0,21464.0,53139.0,40208.0,45367.0,37655.0
...,...,...,...,...,...,...,...,...,...
141915,NaN,Centraal persoonsregister (CPR),2015,NaN,NaN,NaN,NaN,NaN,NaN
141916,NaN,Centraal persoonsregister (CPR),2016,NaN,NaN,NaN,NaN,NaN,NaN
141917,NaN,Centraal persoonsregister (CPR),2017,NaN,NaN,NaN,NaN,NaN,NaN
141918,NaN,Centraal persoonsregister (CPR),2018,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
filters = ["Nationaliteit eq 'NAT9489'", # total non-dutch
           "Nationaliteit eq 'NAT9275'", # total EU excl. Dutch
           "Nationaliteit eq 'NAT9482'", # total Western excl. Dutch
           "Nationaliteit eq 'NAT9483'"] # total Non-Western

data = pd.DataFrame()
for x in filters:
    data = data.append(pd.DataFrame(cbs.get_data('84974NED',
                                          select = ['Geslacht', 'RegioS', 'Perioden',
                                                    'MigratiesaldoInclusiefAdministratie_3', 'Nationaliteit'],
                                         filters =  x)))

In [53]:
data

,Geslacht,RegioS,Perioden,MigratiesaldoInclusiefAdministratie_3,Nationaliteit
0,Totaal mannen en vrouwen,Nederland,2010,46191.0,Totaal niet-Nederlandse nationaliteit
1,Totaal mannen en vrouwen,Nederland,2011,48243.0,Totaal niet-Nederlandse nationaliteit
2,Totaal mannen en vrouwen,Nederland,2012,34916.0,Totaal niet-Nederlandse nationaliteit
3,Totaal mannen en vrouwen,Nederland,2013,39271.0,Totaal niet-Nederlandse nationaliteit
4,Totaal mannen en vrouwen,Nederland,2014,55905.0,Totaal niet-Nederlandse nationaliteit
...,...,...,...,...,...
185395,Vrouwen,Zwolle,2015,1.0,Niet-westerse nationaliteiten
185396,Vrouwen,Zwolle,2016,-1.0,Niet-westerse nationaliteiten
185397,Vrouwen,Zwolle,2017,0.0,Niet-westerse nationaliteiten
185398,Vrouwen,Zwolle,2018,0.0,Niet-westerse nationaliteiten


In [54]:
val = 'MigratiesaldoInclusiefAdministratie_3'
cat1 = 'Geslacht'
cat2 = 'Nationaliteit'
reg = 'RegioS'
year = 'Perioden'
data = split_column2(data, val, cat1, cat2, reg, year)

In [55]:
data.to_csv('Immigration.csv')

## Downloading total immigrant numbers

alternatives
84910NED - current, from 2010
84727NED - current, from 1995

In [58]:
metadata = pd.DataFrame(cbs.get_meta('84727NED', 'DataProperties'))
metadata

,odata.type,ID,Position,ParentID,Type,Key,Title,Description,MapYear,ReleasePolicy,Datatype,Unit,Decimals,Default,PresentationType
0,Cbs.OData.Dimension,0,0,None,Dimension,Geslacht,Geslacht,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cbs.OData.Dimension,1,1,None,Dimension,Leeftijd,Leeftijd,Het aantal gehele jaren dat op 1 januari van h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cbs.OData.Dimension,2,2,None,Dimension,Nationaliteit,Nationaliteit,Het wettelijk onderdaan zijn van een bepaalde ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cbs.OData.GeoDimension,3,3,None,GeoDimension,RegioS,Regio's,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cbs.OData.TimeDimension,4,4,None,TimeDimension,Perioden,Perioden,,NaN,False,NaN,NaN,NaN,NaN,NaN
5,Cbs.OData.Topic,5,5,None,Topic,Bevolking_1,Bevolking,De inwoners van een bepaald gebied.\r\n\r\nIn ...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute


In [19]:
print(metadata['Description'][2])

Persoon met een migratieachtergrond

Persoon van wie ten minste één ouder in het buitenland is geboren.


In [56]:
filters = ["Migratieachtergrond eq '2012605'", # total non-dutch
           "Migratieachtergrond eq 'H007935'", # total EU excl. Dutch
           "Migratieachtergrond eq '2012655'", # total Western excl. Dutch
           "Migratieachtergrond eq '2012657'"] # total Non-Western

data = pd.DataFrame()
for x in filters:
    data = data.append(pd.DataFrame(cbs.get_data('84910NED',
                                          select = ['Geslacht', 'RegioS', 'Perioden', 'Migratieachtergrond',
                                                    'Generatie', 'BevolkingOp1Januari_1'],
                                         filters =  x)))

In [58]:
# Filter so that only first generation and only data on men and women is recorded
data = data[(data['Generatie'] == '1e generatie migratieachtergrond') & 
             ((data['Geslacht'] == 'Mannen') | 
             (data['Geslacht'] == 'Vrouwen'))].drop(['Generatie'], axis=1)

In [60]:
val = 'BevolkingOp1Januari_1'
cat1 = 'Geslacht'
cat2 = 'Migratieachtergrond'
reg = 'RegioS'
year = 'Perioden'
data = split_column2(data, val, cat1, cat2, reg, year)

In [61]:
data.to_csv('Total_Immigration.csv')

## Downloading Income Data

alternatives:
80366ned is archive data, only has mean, from 2005 to 2014
84867NED is current, has mean and median from 2011
80500ned is archive, has mean and median from 2005 to 2014

In [24]:
metadata = pd.DataFrame(cbs.get_meta('80500ned', 'DataProperties'))
metadata

,odata.type,ID,Position,ParentID,Type,Key,Title,Description,MapYear,ReleasePolicy,Datatype,Unit,Decimals,Default,PresentationType
0,Cbs.OData.Dimension,0,0.0,NaN,Dimension,Populatie,Populatie,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cbs.OData.Dimension,1,1.0,NaN,Dimension,Persoonskenmerken,Persoonskenmerken,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cbs.OData.GeoDimension,2,2.0,NaN,GeoDimension,RegioS,Regio's,,2011.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Cbs.OData.TimeDimension,3,3.0,NaN,TimeDimension,Perioden,Perioden,None,NaN,False,NaN,NaN,NaN,NaN,NaN
4,Cbs.OData.TopicGroup,4,NaN,NaN,TopicGroup,,Personen met inkomen,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cbs.OData.Topic,5,4.0,4.0,Topic,AantalPersonen_1,Aantal personen,"Een persoon heeft inkomen, indien er sprake is...",NaN,NaN,Double,x 1 000,1.0,Impossible,Absolute
6,Cbs.OData.Topic,6,5.0,4.0,Topic,RelatiefAantalPersonen_2,Relatief aantal personen,Aantal personen met (heel jaar) inkomen in pro...,NaN,NaN,Double,%,1.0,Impossible,Relative
7,Cbs.OData.Topic,7,6.0,NaN,Topic,GemiddeldPersoonlijkInkomen_3,Gemiddeld persoonlijk inkomen,Het persoonlijk inkomen omvat de volgende best...,NaN,NaN,Double,1 000 euro,1.0,Impossible,Relative
8,Cbs.OData.Topic,8,7.0,NaN,Topic,MediaanPersoonlijkInkomen_4,Mediaan persoonlijk inkomen,Het mediane inkomen is gelijk aan het middelst...,NaN,NaN,Double,1 000 euro,1.0,Impossible,Relative


In [15]:
print(metadata['Description'][4])

Met ingang van het RIO 2010 zijn de resultaten gebaseerd op integrale waarneming. De eerder gepubliceerde resultaten van het RIO (2005 – 2009 voor methodewijziging) zijn gebaseerd op een steekproef van ca. 2 miljoen huishoudens.
Om vergelijking tussen 2010 integraal en 2009 steekproef mogelijk te maken zijn voor 2009 na methodewijziging eveneens op basis van integrale waarneming uitkomsten gepubliceerd waardoor voor het jaar 2009 (breukjaar) zowel uitkomsten op basis van steekproef als integrale waarneming zijn opgenomen.



In [62]:
data = pd.DataFrame(cbs.get_data('84867NED',
                                      select = ['Geslacht', 'RegioS', 'Perioden',
                                                'Persoonskenmerken', 'GemiddeldGestandaardiseerdInkomen_4',
                                                'MediaanGestandaardiseerdInkomen_5'],
                                     filters = "Persoonskenmerken eq 'T009002'"))


In [63]:
val = ['GemiddeldGestandaardiseerdInkomen_4', 'MediaanGestandaardiseerdInkomen_5']
cat = 'Geslacht'
reg = 'RegioS'
year = 'Perioden'
data = split_column(data, val, cat, reg, year)

In [65]:
data.to_csv('Income.csv')

## Downloading Marriage Data

In [13]:
metadata = pd.DataFrame(cbs.get_meta('37890', 'DataProperties'))
metadata

,odata.type,ID,Position,ParentID,Type,Key,Title,Description,MapYear,ReleasePolicy,Datatype,Unit,Decimals,Default,PresentationType
0,Cbs.OData.GeoDimension,0,0.0,NaN,GeoDimension,RegioS,Regio's,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cbs.OData.TimeDimension,1,1.0,NaN,TimeDimension,Perioden,Perioden,None,NaN,False,NaN,NaN,NaN,NaN,NaN
2,Cbs.OData.TopicGroup,2,NaN,NaN,TopicGroup,,Huwen,Trendbreuk huwelijkssluitingen en huwende pers...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cbs.OData.TopicGroup,3,NaN,2.0,TopicGroup,,Huwelijkssluitingen,Trendbreuk huwelijkssluitingen en huwende pers...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cbs.OData.Topic,4,2.0,3.0,Topic,TotaalHuwelijkssluitingen_1,Totaal huwelijkssluitingen,None,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
5,Cbs.OData.Topic,5,3.0,3.0,Topic,BeidenWonenInHuwelijksgemeente_2,Beiden wonen in huwelijksgemeente,Huwelijksluitingen waarvan beide partners staa...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
6,Cbs.OData.Topic,6,4.0,3.0,Topic,AlleenManWoontInHuwelijksgemeente_3,Alleen man woont in huwelijksgemeente,Huwelijkssluitingen waarvan alleen de man staa...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
7,Cbs.OData.Topic,7,5.0,3.0,Topic,AlleenVrouwWoontInHuwelijksgemeente_4,Alleen vrouw woont in huwelijksgemeente,Huwelijkssluitingen waarvan alleen de vrouw st...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
8,Cbs.OData.Topic,8,6.0,3.0,Topic,BeidenWonenNietInHuwelijksgemeente_5,Beiden wonen niet in huwelijksgemeente,Huwelijkssluitingen waarvan beide partners NIE...,NaN,NaN,Long,aantal,0.0,Impossible,Absolute
9,Cbs.OData.TopicGroup,9,NaN,2.0,TopicGroup,,Huwende personen,Trendbreuk huwelijkssluitingen en huwende pers...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print(metadata['Description'][11])

Huwende mannen per duizend van het gemiddeld aantal niet gehuwde mannen van 15 jaar of ouder, gestandaardiseerd naar leeftijd.

Standaardisatie:
Bij standaardisatie wordt de invloed van de bevolking naar leeftijd uitgeschakeld. De functie van standaardisatie is het beter vergelijkbaar maken van cijfers tussen de verschillende gemeenten, regio's en Nederland. 
De bevolking van Nederland als geheel heeft als standaardbevolking gediend.

Leeftijd:
Het aantal gehele jaren dat op 31 december in het jaar van huwelijkssluiting is verstreken sinds de geboortedatum van de persoon. Te berekenen als jaar van huwelijkssluiting min geboortejaar.

Gemiddeld aantal niet-gehuwde mannen:
De helft van het aantal niet-gehuwde mannen met een bepaald geboortejaar aan het begin van een periode plus de helft van de niet-gehuwde mannen met dat zelfde geboortejaar aan het einde van de periode.


In [49]:
data = pd.DataFrame(cbs.get_data('37890',
                                      select = ['RegioS', 'Perioden',
                                                'TotaalHuwelijkssluitingen_1', 'HuwendeMannenRelatief_7',
                                                'HuwendeVrouwenRelatief_9']))

# Making RegioS and Perioden the index values as in other dataframes

data.index = pd.MultiIndex.from_arrays(data[['RegioS', 'Perioden']].values.T, names=['RegioS', 'Perioden'])
data = data.drop(['RegioS', 'Perioden'], axis=1)

data.to_csv('Marriage.csv')

## Downloading Crime Data

alternatives:
83648NED is current, very detailed crime, also per 1000 people

83223NED is archive, less detailed, but still ok, problem: only has registered cases with at least 1 suspect 

In [9]:
metadata = pd.DataFrame(cbs.get_meta('83648NED', 'RegioS'))
metadata

,Key,Title,Description,CategoryGroupID
0,NL01,Nederland,,None
1,LD01,Noord-Nederland (LD),LD = Landsdeel\r\n\r\nNoord-Nederland: Groning...,None
2,LD02,Oost-Nederland (LD),LD = Landsdeel\r\n\r\nOost-Nederland: Overijss...,None
3,LD03,West-Nederland (LD),"LD = Landsdeel\r\n\r\nWest-Nederland: Utrecht,...",None
4,LDG4,"West-Nederland, exclusief G4 (LD)","LD = Landsdeel\r\n\r\nWest-Nederland: Utrecht,...",None
5,LD04,Zuid-Nederland (LD),LD = Landsdeel\r\n\r\nZuid-Nederland: Noord-Br...,None
6,LD99,Niet in te delen (LD),LD = Landsdeel\r\n\r\nRegionale groepering van...,None
7,PV20,Groningen (PV),PV = Provincie\r\n\r\nBestuurlijke onderverdel...,None
8,PV21,Fryslân (PV),PV = Provincie\r\n\r\nBestuurlijke onderverdel...,None
9,PV22,Drenthe (PV),PV = Provincie\r\n\r\nBestuurlijke onderverdel...,None


In [22]:
print(metadata['Description'][3])

Een ingeschreven (rechtbank)strafzaak is een bij het parket ingeschreven dossier met het proces-verbaal van aangifte ten aanzien van één verdachte. Als er meer verdachten in een proces-verbaal voorkomen wordt voor iedere verdachte een strafzaak ingeschreven.


In [36]:
# for 83223NED
data = pd.DataFrame(cbs.get_data('83223NED',
                                      select = ['Regiokenmerken', 'Perioden',
                                                'Misdrijven', 'IngeschrevenRechtbankstrafzaken_1']))

In [66]:
# for 83648NED
filters = ["SoortMisdrijf eq 'CRI1000'",
           "SoortMisdrijf eq 'CRI2000'",
           "SoortMisdrijf eq 'CRI3000'",
           "SoortMisdrijf eq 'CRI4000'",
           "SoortMisdrijf eq 'CRI5000'",
           "SoortMisdrijf eq 'CRI6000'",
           "SoortMisdrijf eq 'CRI7000'",
           "SoortMisdrijf eq 'CRI9000'"]

data = pd.DataFrame()
for x in filters:
    data = data.append(pd.DataFrame(cbs.get_data('83648NED',
                                          select = ['RegioS', 'Perioden',
                                                    'GeregistreerdeMisdrijvenPer1000Inw_3', 'SoortMisdrijf'],
                                         filters =  x)))



In [67]:
val = 'GeregistreerdeMisdrijvenPer1000Inw_3'
cat = 'SoortMisdrijf'
reg = 'RegioS'
year = 'Perioden'
data = split_column(data, val, cat, reg, year)

In [68]:
data.to_csv('Crime.csv')

## Downloading Data Number of workers in industries

For number of total workers:
currently in dataset: 83582NED - from 2010 to 2019


addition 1: 70742ned - from 1993 to 2005

addition 2: 81402ned - from 2008 to 2014,

ACTUALLY, use this one: 84419NED
- we now need work volume, not number of jobs

For number of workers in industry at beginning of timeframe: 70090ENG

### Workers in industries

In [11]:
metadata = pd.DataFrame(cbs.get_meta('70090NED', 'RegioS'))
metadata

,Key,Title,Description,CategoryGroupID
0,NL1,Nederland,None,None
1,ExTR,Extra-territoriale regio,"Omvat de territoriale wateren, het Nederlandse...",None
2,LD01,Noord-Nederland,"Omvat de provincies Groningen, Friesland en Dr...",None
3,LD02,Oost-Nederland,"Omvat de provincies Gelderland, Overijssel en ...",None
4,LD03,West-Nederland,"Omvat de provincies Noord-Holland, Zuid-Hollan...",None
5,LD04,Zuid-Nederland,Omvat de provincies Noord-Brabant en Limburg.,None
6,PV20,Groningen (pv),Omvat de volgende COROP-gebieden Oost-Groninge...,None
7,PV21,Friesland (pv),Omvat de volgende COROP-gebieden Noord-Friesla...,None
8,PV22,Drenthe (pv),Omvat de volgende COROP-gebieden Noord-Drenthe...,None
9,PV23,Overijssel (pv),Omvat de volgende COROP-gebieden Noord-Overijs...,None


In [6]:
print(metadata['Description'][15])

Omvat de volgende COROP-gebieden Zeeuwsch-Vlaanderen en Overig Zeeland.
I.v.m. de geheimhouding is samengenomen:
SBI 28 Metaalproductenindustrie en SBI DK Machine-industrie bij
SBI 27 Basismetaalindustrie.


In [16]:
# Only get data for provinces and corop regions
reglist = [f"RegioS eq '{x}'" for x in metadata.Key.tolist() if x.startswith('PV') or x.startswith('CR') or x.startswith('NL')]

data = pd.DataFrame()
for y in reglist:
    data = data.append(pd.DataFrame(cbs.get_data('70090NED',
                                          select = ['RegioS', 'Perioden','SBI93','Arbeidsvolume_2'],
                                         filters =  y)))
    
# Transform string value in Perioden into int
data['Perioden'] = data['Perioden'].replace(to_replace = ' \w+', value='', regex=True).astype(int)

# Only data from Industry and all industries + only 2008 and 2009
data = data[data['SBI93'].str.contains(r'(^2)|(^D[A-Z])|(Overige)|(Totaal bedrijfsactiviteiten)')]

# Change (pv) to (PV)
data.RegioS = data.RegioS.str.replace('\(pv\)', '(PV)', regex=True)

# Change w/o (PV) to (CR)
data.loc[~data.RegioS.str.startswith('Nederland') & ~data.RegioS.str.endswith('(PV)'),['RegioS']] = data.loc[data.RegioS.str.endswith('(PV)') & ~data.RegioS.str.endswith('(PV)')]['RegioS'] + ' (CR)'

/home/hennes/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [17]:
# Turn dataset wide

val = 'Arbeidsvolume_2'
cat = 'SBI93'
reg = 'RegioS'
year = 'Perioden'
data = split_column(data, val, cat, reg, year)

In [19]:
data = data.rename(columns={'DA Voedings- en genotmiddelen_Arbeidsvolume_2':'labour_food, beverages, tobacco',
                           'DB+DC Textiel en leder_Arbeidsvolume_2':'labour_textile, clothes, leather',
                           '21 Papier en karton_Arbeidsvolume_2':'labour_paper, carton',
                           '22 Uitgeverijen en drukkerijen_Arbeidsvolume_2':'labour_printing, recorded media',
                           'DF Aardolie- en steenkool industrie_Arbeidsvolume_2':'labour_coke_and_refined_petrolium',
                           'DG+DH Chemische, rubber en kunststof ind_Arbeidsvolume_2':'labour_chemicals, chemical products, non-metallic products',
                           '27 Basismetaalindustrie_Arbeidsvolume_2':'labour_basic_metals',
                           '28+DK Metaalproducten en machine ind._Arbeidsvolume_2':'labour_metal_products,_machinery_and_equipment',
                           'DL Elektrotechnische industrie_Arbeidsvolume_2':'labour_electronic_products',
                           'DM Transportmiddelen industrie_Arbeidsvolume_2':'labour_transport_vehicles',
                           'Overige industrie_Arbeidsvolume_2':'labour_other industrial products',
                           'Totaal bedrijfsactiviteiten_Arbeidsvolume_2':'labour_all_industries'})

labour_food, beverages, tobacco --> 'import_food_products', 'import_beverages', 'import_tobacco_products',
labour_textile, clothes, leather --> 'import_textiles', 'import_wearing_apparel', 'import_leather_and_leather-related_products'

labour_paper, carton --> import_paper_and_related_products
labour_printing, recorded media --> import_printing_and_recorded_media
labour_coke_and_refined_petrolium --> import_coke_and_refined_petrolium
labour_chemicals, chemical products, non-metallic products --> 'import_chemicals_and_chemical_products', import_basic_pharmaceutical_products_and_preparations',  'import_rubber_and_plastic_products',

labour_basic_metals --> import_basic_metals
labour_metal_products,_machinery_and_equipment --> 'import_fabricated_metal_products',  'import_machinery_and_equipment'

labour_electronic_products -->  'import_computer_electronic_and_optical_products', 'import_electrical_equipment',
labour_transport_vehicles -->  'import_motor_vehicles_trailers_and_semi-trailers','import_other_transport_equipment'

labour_other industrial products --> import_wood_and_wood_products, 'import_other_non-metallic_products',  'import_furniture', 'import_other'

In [24]:
data.to_csv('Workers.csv')

## Downloading Data for change in Chinese Imports

In [176]:
metadata = pd.DataFrame(cbs.get_meta('83028NED', 'SITC'))
metadata

,Key,Title,Description,CategoryGroupID
0,T001082,Totaal goederen,,None
1,A018524,0 Voeding en levende dieren,,None
2,A018528,1 Dranken en tabak,,None
3,A018540,"2 Grondstoffen, niet eetbaar, behalve...","2 Grondstoffen, niet eetbaar, behalve brandsto...",None
4,A018591,"3 Minerale brandstoffen, smeermiddele...","3 Minerale brandstoffen, smeermiddelen en derg...",None
5,A018612,4 Dierlijke en plantaardige oliën en ...,4 Dierlijke en plantaardige oliën en vetten,None
6,A018629,5 Chemische producten,,None
7,A018678,6 Fabricaten hoofdzakelijk gerangschi...,6 Fabricaten hoofdzakelijk gerangschikt volgen...,None
8,A018743,7 Machines en vervoermaterieel,,None
9,A018809,8 Diverse gefabriceerde goederen,,None


In [110]:
print(metadata['Description'][7])

6 Fabricaten hoofdzakelijk gerangschikt volgens grondstoffen


In [101]:
data = pd.DataFrame(cbs.get_data('83028NED',
                                          select = ['SITC', 'Perioden','Invoerwaarde_1', 'LandenGroepen'],
                                          filters = "LandenGroepen eq 'L008575'"))

data = data[(data['Perioden'].str.match(r'(^\d{4}$)|(^\d{4} $)'))]
data['Perioden'] = data['Perioden'].astype(int)

In [99]:
for x in data[f'SITC'].unique():
            data[f'{x}_Invoerwaarde_1'] = np.where(data[f'SITC'] == x, data[f'Invoerwaarde_1'], None)
data = data.drop([f'Invoerwaarde_1'], axis=1)
data['RegioS'] = 'Nederland'
data = data.groupby(['RegioS', 'Perioden']).agg(max).drop([f'SITC'], axis=1)

In [100]:
data.to_csv('Imports.csv')

## Importing Election data

In [43]:
for file in glob('Elections/*'):
    data = pd.read_csv(file, sep=';')
    data.iloc[:, 10:] = data.iloc[:, 10:].div(data.GeldigeStemmen, axis=0) # transform absolute votes into relative share
    data['RegioS'] = data['RegioNaam']
    year = re.search(r'\d{4}',file).group(0)
    data['Perioden'] = year
    data['turnout'] = data.Opkomst/data.Kiesgerechtigden
    data.set_index(['RegioS', 'Perioden'], inplace=True)
    data = data.iloc[:, 10:]
    results = data.iloc[:, :-3]
    results = results[results.columns[(results>0.05).any()]]
    results['turnout'] = data['turnout']
    results.to_csv(f'election_{year}.csv')

In [44]:
data = pd.read_csv('election_1989.csv')
if 'Democraten 66 (D66)' in data.columns:
    data = data.rename(columns={'Democraten 66 (D66)':'D66'})
if 'Partij van de Arbeid (P.v.d.A.)' in data.columns:
    data = data.rename(columns={'Partij van de Arbeid (P.v.d.A.)':'PvdA'})
if 'Staatkundig Gereformeerde Partij (SGP)' in data.columns:
    data = data.rename(columns={'Staatkundig Gereformeerde Partij (SGP)':'SGP'})
if 'GROENLINKS' in data.columns:
    data = data.rename(columns={'GROENLINKS':'GL'})
for file in glob('*'):
    if (file.startswith('election')) and (file != 'election_1989.csv'):
        df = pd.read_csv(file)
        if 'Democraten 66 (D66)' in df.columns:
            df = df.rename(columns={'Democraten 66 (D66)':'D66'})
        if 'Partij van de Arbeid (P.v.d.A.)' in df.columns:
            df = df.rename(columns={'Partij van de Arbeid (P.v.d.A.)':'PvdA'})
        if 'Staatkundig Gereformeerde Partij (SGP)' in df.columns:
            df = df.rename(columns={'Staatkundig Gereformeerde Partij (SGP)':'SGP'})
        if 'GROENLINKS' in df.columns:
            df = df.rename(columns={'GROENLINKS':'GL'})
        data = data.merge(df, how='outer')

In [45]:
data = data.sort_values(by=['RegioS', 'Perioden']).set_index(['RegioS', 'Perioden'])

In [46]:
data.to_csv('elections.csv')
for file in glob('*'):
    if (file.startswith('election_')):
        os.remove(file)

## Getting Gemeentelijke Indeeling 

### NUTS3

In [17]:
# I will do it instead with the Table List

In [18]:
import pprint

In [19]:
# Get dictionary with year dataset identifiers and matching year number

tables = [x for x in cbs.get_table_list() if x['Title'].startswith('Gebieden in Nederland')]
table_dict = dict(zip([re.search(r'\d{4}' ,x['Period']).group() for x in tables], [x['Identifier'] for x in tables]))

In [20]:
# Create dict withformat: dataset code: [year, variable name]
dict_combis = {}
for x, y in table_dict.items():
    metadata = pd.DataFrame(cbs.get_meta(y, 'DataProperties'))
    key = metadata[metadata['Description'].str.contains(r'Namen van de (COROP|Corop)-gebieden',
                                                        na=False)].Key.tolist()[0]
    dict_combis.update({x:[y, key]})


/home/hennes/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [21]:
# download gemeente NUTS3 translations for every year

df = []
for x, y in dict_combis.items():
    if int(x) > 2002:
        data = pd.DataFrame(cbs.get_data(y[0], select = [y[1], 'RegioS']))
        data['Perioden'] = x
        data.rename(columns={f'{y[1]}': 'nuts3'}, inplace=True)
        df.append(data)
    else:
        data = pd.DataFrame(cbs.get_data(y[0], select = [y[1], 'Gemeenten']))
        data['Perioden'] = x
        data.rename(columns={'Gemeenten':'RegioS', f'{y[1]}': 'nuts3'}, inplace=True)
        df.append(data)
df = pd.concat(df)
df.to_csv('nuts3.csv')

### NUTS2

In [22]:
import pprint

In [23]:
# Get dictionary with year dataset identifiers and matching year number

tables = [x for x in cbs.get_table_list() if x['Title'].startswith('Gebieden in Nederland')]
table_dict = dict(zip([re.search(r'\d{4}' ,x['Period']).group() for x in tables], [x['Identifier'] for x in tables]))

In [24]:
# Create dict withformat: dataset code: [year, variable name]
dict_combis = {}
for x, y in table_dict.items():
    metadata = pd.DataFrame(cbs.get_meta(y, 'DataProperties'))
    key = metadata[metadata['Description'].str.contains(r'Namen van de provincies',
                                                        na=False)].Key.tolist()[0]
    dict_combis.update({x:[y, key]})


In [25]:
# download gemeente NUTS2 translations for every year

df = []
for x, y in dict_combis.items():
    if int(x) > 2002:
        data = pd.DataFrame(cbs.get_data(y[0], select = [y[1], 'RegioS']))
        data['Perioden'] = x
        data.rename(columns={f'{y[1]}': 'nuts2'}, inplace=True)
        df.append(data)
    else:
        data = pd.DataFrame(cbs.get_data(y[0], select = [y[1], 'Gemeenten']))
        data['Perioden'] = x
        data.rename(columns={'Gemeenten':'RegioS', f'{y[1]}': 'nuts2'}, inplace=True)
        df.append(data)
df = pd.concat(df)
df.to_csv('nuts2.csv')